In [10]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V=[{}]
    for i in states:
        V[0][i]=start_p[i]*emit_p[i][obs[0]]
    # Run Viterbi when t > 0
    print("Init V: ", V)
    for t in range(1, len(obs)):
        V.append({})
        for y in states:
            (prob, state) = max((V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]], y0) for y0 in states)
            V[t][y] = prob
            print(state)
        for i in dptable(V):
            print (i)
        opt=[]
        for j in V:
            for x,y in j.items():
                if j[x]==max(j.values()):
                    opt.append(x)
    #the highest probability
    print("This is V: ", V)
    print("This is opt: ", opt)
    h=max(V[-1].values())
    print ('The steps of states are '+' '.join(opt)+' with highest probability of %s'%h)
    #it prints a table of steps from dictionary

def dptable(V):
    yield " ".join(("%10d" % i) for i in range(len(V)))
    for y in V[0]:
        yield "%.7s: " % y+" ".join("%.7s" % ("%f" % v[y]) for v in V)

states = ('Healthy', 'Fever')
observations = ('normal', 'cold', 'dizzy')
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
transition_probability = {
'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
}
emission_probability = {
'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
}

viterbi(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability)

Init V:  [{'Healthy': 0.3, 'Fever': 0.04000000000000001}]
Healthy
Healthy
         0          1
Healthy: 0.30000 0.08400
Fever: 0.04000 0.02700
Healthy
Healthy
         0          1          2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
This is V:  [{'Healthy': 0.3, 'Fever': 0.04000000000000001}, {'Healthy': 0.084, 'Fever': 0.027}, {'Healthy': 0.00588, 'Fever': 0.01512}]
This is opt:  ['Healthy', 'Healthy', 'Fever']
The steps of states are Healthy Healthy Fever with highest probability of 0.01512


O1 = S0 XOR S1 XOR I
O2 = S0 XOR S2
S1, S2 = Next stage

In [51]:
#inputs = [1, 1, 1, 1]
inputs = '00110'


def xor(bit0, bit1):
    if (bit0 == bit1):
        return '0'
    else:
        return '1'


def viterbi_encoder(inputs):
    if (type(inputs) is int):
        inputs = str(inputs)
    else:
        inputs = ''.join(map(str, inputs))
    print(inputs)
    #shift register encoder
    s_reg = ["0", "0", "0"]
    obs = []
    for t in range(0, len(inputs)):
        #shifting the bits to right
        s_reg[2] = s_reg[1]
        s_reg[1] = s_reg[0]
        #inserting input
        s_reg[0] = inputs[t]
        state = s_reg[0] + s_reg[1]
        obs.append([])
        #encoded bits
        obs[t] = xor(xor(s_reg[0], s_reg[1]), s_reg[2]) + xor(
            s_reg[0], s_reg[2])
        print(s_reg, "State: " + state, "Output: " + obs[t])
    print(obs)


viterbi_encoder(inputs)

00110
['0', '0', '0'] State: 00 Output: 00
['0', '0', '0'] State: 00 Output: 00
['1', '0', '0'] State: 10 Output: 11
['1', '1', '0'] State: 11 Output: 01
['0', '1', '1'] State: 01 Output: 01
['00', '00', '11', '01', '01']


In [55]:
obs = ("11", "10", "11", "11", "01", "01", "11")
start_metric = {'00': 0, '01': 0, '10': 0, '11': 0}
state_machine = {
    #current state, possible branches, branch information
    '00': {
        'b1': {
            'out_b': "11",
            'prev_st': '01',
            'input_b': 0
        },
        'b2': {
            'out_b': "00",
            'prev_st': '00',
            'input_b': 0
        }
    },
    '01': {
        'b1': {
            'out_b': "01",
            'prev_st': '11',
            'input_b': 0
        },
        'b2': {
            'out_b': "10",
            'prev_st': '10',
            'input_b': 0
        }
    },
    '10': {
        'b1': {
            'out_b': "11",
            'prev_st': '00',
            'input_b': 1
        },
        'b2': {
            'out_b': "00",
            'prev_st': '01',
            'input_b': 1
        }
    },
    '11': {
        'b1': {
            'out_b': "10",
            'prev_st': '11',
            'input_b': 1
        },
        'b2': {
            'out_b': "01",
            'prev_st': '10',
            'input_b': 1
        }
    },
}


def bits_diff_num(num_1, num_2):
    count = 0
    for i in range(0, len(num_1), 1):
        if num_1[i] != num_2[i]:
            count = count + 1
    return count


def viterbi(obs, start_metric, state_machine):
    #Trellis structure
    V = [{}]
    for st in state_machine:
        # Calculating the probability of both initial possibilities for the first observation
        V[0][st] = {"metric": start_metric[st]}
    #for t&amp;amp;amp;amp;amp;amp;amp;amp;gt;0
    for t in range(1, len(obs) + 1):
        V.append({})
        for st in state_machine:
            #Check for smallest bit difference from possible previous paths, adding with previous metric
            prev_st = state_machine[st]['b1']['prev_st']
            first_b_metric = V[(t - 1)][prev_st]["metric"] + bits_diff_num(
                state_machine[st]['b1']['out_b'], obs[t - 1])
            prev_st = state_machine[st]['b2']['prev_st']
            second_b_metric = V[(t - 1)][prev_st]["metric"] + bits_diff_num(
                state_machine[st]['b2']['out_b'], obs[t - 1])
            #print(state_machine[st]['b1']['out_b'],obs[t - 1],t)
            if first_b_metric > second_b_metric:
                V[t][st] = {"metric": second_b_metric, "branch": 'b2'}
            else:
                V[t][st] = {"metric": first_b_metric, "branch": 'b1'}

    for st in state_machine:
        for t in range(0, len(V)):
            print(V[t][st]["metric"], end='')
        print("")
    print("")

    smaller = min(V[t][st]["metric"] for st in state_machine)
    #traceback the path on smaller metric on last trellis column
    for st in state_machine:
        if V[len(obs) - 1][st]["metric"] == smaller:
            source_state = st
            for t in range(len(obs), 0, -1):
                branch = V[t][source_state]["branch"]
                print(state_machine[source_state][branch]['input_b']),
                source_state = state_machine[source_state][branch]['prev_st']
            print(source_state + "\n")
    print("Finish")


viterbi(obs, start_metric, state_machine)

00102330
01022203
00110332
01122023

0
1
1
1
0
0
1
00

Finish
